In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
from queue import Queue
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
import requests

In [103]:
# Função para acessar o portal com os parametros desejados
def consultar(navegador):
    url='https://transparencia.mprj.mp.br/web/novo-portal-transparencia/processos-distribuidos-novo'
    navegador.get(url)

In [216]:
# Iniciando drive do Chorme

op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)
# driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver)
sleep(1)

In [217]:
# Identificadores do HTML

comarca_id = 'comarcaId'
orgao_id = 'orgaoId'
tipo_documento_id = 'documentoId'
membro_id = 'membroId'

btn_buscar_id = 'export'

linha_tabela_class = 'tr-modal' #'tabela-cargos-membros'
pagination_css = 'li.page-item.active'

def get_element(id):
    element = driver.find_element(By.ID, id)
    return element


In [218]:
from datetime import datetime

def save_to_csv(name, col_comarca, col_orgao, col_membro, col_numero, col_distribuido_em, col_submetido_em, col_tipo, col_unidade):
    file_name = f'{name}_{datetime.now().strftime("%d-%m-%Y_%H-%M-%S")}'
    scraping_values = {'Comarca': col_comarca, 'Orgão de execução': col_orgao, 'Tipo de documento': col_tipo, 'Membro': col_membro, 'Número': col_numero, 'Distribuído em': col_distribuido_em, 'Submetido em': col_submetido_em, 'Tipo': col_tipo, 'Unidade': col_unidade}
    df = pd.DataFrame.from_dict(scraping_values) 

    df.to_csv(f'{file_name}.csv')

In [219]:
comarcas_visitados = []

comarcas = get_element(comarca_id)

for comarca in comarcas.find_elements(By.TAG_NAME, "option"):

    # manualmente terminei o CAMPOS DOS GOYTACAZES

    for i in range(0, 15):
        comarcas_visitados.append(comarcas.find_elements(By.TAG_NAME, "option")[i].get_attribute('innerHTML'))

    if comarca.get_attribute('innerHTML') in comarcas_visitados:
        continue

    col_comarca = []
    col_orgao = []
    col_tipo_doc = []
    col_membro = []
    col_numero = []
    col_distribuido_em = []
    col_submetido_em = []
    col_tipo = []
    col_unidade = []   
     
    value = comarca.get_attribute('value')

    if value:
        Select(comarcas).select_by_value(value)
    else:
        continue

    name_comarca = comarca.text
    sleep(1)

    orgaos = get_element(orgao_id)
    for orgao in orgaos.find_elements(By.TAG_NAME, "option"):
        value = orgao.get_attribute('value')
        if value:
            Select(orgaos).select_by_value(value)  
        else:
            continue   

        name_orgao = orgao.text
        sleep(1)

        tipos = get_element(tipo_documento_id)
        for tipo in tipos.find_elements(By.TAG_NAME, "option"):
            value = tipo.get_attribute('value')
            if value:
                Select(tipos).select_by_value(value)  
            else:
                continue    

            name_tipo_doc = tipo.text
            sleep(1)

            membros = get_element(membro_id)
            for membro in membros.find_elements(By.TAG_NAME, "option"):
                value = membro.get_attribute('value')
                if value:
                    Select(membros).select_by_value(value)     
                    # print('membro encontrado')
                else:
                    # print('membro nao encontrado')
                    continue

                name_membro = membro.text
                                    
                # Busca
                buscar = get_element(btn_buscar_id).click() 
   
                sleep(1)          

                pages_visited = []

                actual_page = driver.find_element(By.CSS_SELECTOR, pagination_css)   
                # pages_visited.append(actual_page.text)

                # next_pages = actual_page.find_elements(By.XPATH, './/following-sibling::li')

                # for i in range(0, len(next_pages)):
                while True:
                    # print(next_pages[i].get_attribute('innerHTML'))

                    # if next_pages[i].text in pages_visited:
                    #     continue
                    linhas_tabela = driver.find_elements(By.CLASS_NAME, linha_tabela_class)  

                    for linha in linhas_tabela:     
                        # print(linha.get_attribute('innerHTML'))
                        
                        try:
                            colunas = linha.find_elements(By.TAG_NAME, 'td')
                        except Exception:
                            break

                        colunas = list(colunas)

                        # print(list(colunas)[0].text)
                        # print(colunas[1].get_attribute('text'))

                        # print(len(linhas_tabela))

                        col_numero.append(colunas[0].text)
                        col_distribuido_em.append(colunas[1].text)
                        col_submetido_em.append(colunas[2].text)
                        col_tipo.append(colunas[3].text)
                        col_unidade.append(colunas[4].text)   
                        col_comarca.append(name_comarca)
                        col_orgao.append(name_orgao)
                        col_tipo_doc.append(name_tipo_doc)
                        col_membro.append(name_membro)

                # pages_visited.append(next_pages[i].text)
                    # next_pages[i].click()         
                    next_page = actual_page.find_elements(By.XPATH, './/following-sibling::li')[-1]

                    try:
                        next_page.find_element(By.TAG_NAME, 'a') 
                    except Exception:
                        # Não tem próxima pagina
                        break

                    next_page.click()   
                    sleep(1)    

                driver.find_element(By.CSS_SELECTOR, 'button.close.btn.btn-unstyled').click() #fechar modal

    save_to_csv(name_comarca, col_comarca, col_orgao, col_membro, col_numero, col_distribuido_em, col_submetido_em, col_tipo, col_unidade)
    comarcas_visitados.append(comarca.get_attribute('innerHTML'))

ElementClickInterceptedException: Message: element click intercepted: Element <li class="page-item">...</li> is not clickable at point (588, 523). Other element would receive the click: <div class="modal-backdrop modal-loader-back fade show"></div>
  (Session info: chrome-headless-shell=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7D93BEEA2+31554]
	(No symbol) [0x00007FF7D9337ED9]
	(No symbol) [0x00007FF7D91F872A]
	(No symbol) [0x00007FF7D925012E]
	(No symbol) [0x00007FF7D924DAF2]
	(No symbol) [0x00007FF7D924AF8B]
	(No symbol) [0x00007FF7D924A156]
	(No symbol) [0x00007FF7D923C151]
	(No symbol) [0x00007FF7D926D02A]
	(No symbol) [0x00007FF7D923BA76]
	(No symbol) [0x00007FF7D926D240]
	(No symbol) [0x00007FF7D928C977]
	(No symbol) [0x00007FF7D926CDD3]
	(No symbol) [0x00007FF7D923A33B]
	(No symbol) [0x00007FF7D923AED1]
	GetHandleVerifier [0x00007FF7D96C8B1D+3217341]
	GetHandleVerifier [0x00007FF7D9715AE3+3532675]
	GetHandleVerifier [0x00007FF7D970B0E0+3489152]
	GetHandleVerifier

In [221]:
comarcas_visitados = []

# manualmente terminei o CANTAGALO

for i in range(0, 15):
    comarcas_visitados.append(comarcas.find_elements(By.TAG_NAME, "option")[i].get_attribute('innerHTML'))

comarcas_visitados

['',
 'ANGRA DOS REIS',
 'ARARUAMA',
 'ARMAÇÃO DOS BÚZIOS',
 'ARRAIAL DO CABO',
 'BARRA DO PIRAÍ',
 'BARRA MANSA',
 'BELFORD ROXO',
 'BOM JARDIM',
 'BOM JESUS DO ITABAPOANA',
 'CABO FRIO',
 'CACHOEIRAS DE MACACU',
 'CAMBUCI',
 'CAMPOS DOS GOYTACAZES',
 'CANTAGALO']

In [186]:
scraping_values = {'Comarca': col_comarca, 'Orgão de execução': col_orgao, 'Tipo de documento': col_tipo, 'Membro': col_membro, 'Número': col_numero, 'Distribuído em': col_distribuido_em, 'Submetido em': col_submetido_em, 'Tipo': col_tipo, 'Unidade': col_unidade}
# scraping_values

In [ ]:
df = pd.DataFrame.from_dict(scraping_values) # columns=['Número', 'Distribuído em', 'Submetido em', 'Tipo', 'Unidade'])


In [139]:
df.to_csv('teste.csv')